install required packages

In [ ]:
import sys
sys.version


In [ ]:
!pip install gpflow==2.5.2
!pip install pyproj
# NOTE: installing cartopy via pip just to avoid import error
# - cartopy should be installed via: conda install -c conda-forge cartopy
# !pip install cartopy


config / parameters


In [ ]:
branch_name = "sparse_dev"
# directory on google drive where to 
# work_sub_dir = ["MyDrive", "workspace"]

mount google drive (use to save results)  - requires login

In [ ]:
import subprocess
import json
from google.colab import drive
import os
import sys


gdrive_mount = '/content/gdrive'
# # requires giving access to google drive account
drive.mount(gdrive_mount)


git pull repository

In [ ]:
import re
# change 'workspace' as needed
# work_dir = os.path.join(gdrive_mount, *["MyDrive", "workspace"])
work_dir = "/content"

# change to working directory
# os.chdir(work_dir)
assert os.path.exists(work_dir), f"workspace directory: {work_dir} does not exist"
os.chdir(work_dir)

# !git clone https://github.com/William-gregory/OptimalInterpolation.git
# url suffix for cloning repp
url = "https://github.com/ronaldBuddys/OptimalInterpolation.git"

# repository directory
repo_dir = os.path.join(work_dir, os.path.basename(url))
repo_dir = re.sub("\.git$", "", repo_dir)

# TODO: put a try except here 
# clone the repo

try:
    git_clone = subprocess.check_output( ["git", "clone", url] , shell=False)
except Exception as e:
    # get non-zero exit status 128: if the repo already exists?
    print(e)

print(f"changing directory to: {repo_dir}")

os.chdir(repo_dir)



Change branch 

In [ ]:
# --
# change branch - review this
# --

try:
    git_checkout = subprocess.check_output(["git", "checkout", "-t", f"origin/{branch_name}"], shell=False)
    print(git_checkout.decode("utf-8") )
except Exception as e:
    git_checkout = subprocess.check_output(["git", "checkout",  f"{branch_name}"], shell=False)
    print(git_checkout.decode("utf-8") )




In [ ]:
# git pull to ensure have the latest
git_pull = subprocess.check_output(["git", "pull"], shell=False)
print(git_pull.decode("utf-8") )

In [ ]:
# add directory to containing repository to sys.path, so can import as a package
# if repo_dir not in sys.path:
#     # tmp_dir = os.path.dirname(repo_dir)
#     print(f"adding {repo_dir} to sys.path")
#     sys.path.extend([])

if work_dir not in sys.path:
    # tmp_dir = os.path.dirname(repo_dir)
    print(f"adding {work_dir} to sys.path")
    sys.path.extend([work_dir])

In [ ]:
# TODO: only downlaod if it does not already exist
import gdown
import zipfile

# print("there was some sort of issue downloading the entire folder structure")
print("will try downloading the zipped version")
# url = "https://drive.google.com/file/d/1c7h6HTT-wbCq_ZKBYLJSSln4tanlLEMZ"
# id = "1c7h6HTT-wbCq_ZKBYLJSSln4tanlLEMZ"


data_dir = os.path.join(repo_dir, "data")
os.makedirs(data_dir, exist_ok=True)

# https://drive.google.com/file/d/1djlaZ2EKbm9pNAEt3w58WJtBA4NyQsNE/view?usp=sharing
id_zip = [
    # {"id": "1ckoowmCwh4tG76sIxXZuVaSSQ0tv8KTU", "zip": "auxiliary.zip", "dirname": "aux"},
    {"id": "1djlaZ2EKbm9pNAEt3w58WJtBA4NyQsNE", "zip": "new_aux.zip", "dirname": "aux"},
    {"id": "1cIh9lskzmL6C7EYV8lmJJ5YaJgKqOZHT", "zip": "CS2S3_CPOM.zip", "dirname": "CS2S3_CPOM"},
    {"id": "1gXsvtxZcWpBALomgeqn9kcfyCtKD3fkz", "zip": "raw_along_track.zip", "dirname": "RAW"},
    # legacy data
    # {"id": "1HcKZD_F3esIPc2NbWlexvXbVOSAlso9m", "zip": "aux_legacy.zip", "dirname": "aux"},
    # {"id": "1Kekh43yTDVJXfSjUrPDV6ZEXXIJhzdXC", "zip": "CS2S3_CPOM_legacy.zip", "dirname": "CS2S3_CPOM"},
]

# TODO: check if output dir already exists: aux and CS2S3_CPOM
for _ in id_zip:
    id = _['id']
    zip = _['zip']
    dirn = _.get('dirname', "")

    if os.path.exists(os.path.join(data_dir, dirn)):
        # print(f"dir{}")
        continue
    # put data in data dir in repository
    output = os.path.join(data_dir, zip)
    gdown.download(id=id, output=output, use_cookies=False)

    # un zip to path
    print("unzipping")
    with zipfile.ZipFile(output, 'r') as zip_ref:
        zip_ref.extractall(path=data_dir)



In [ ]:
import os


In [ ]:
# change to repo_dir so can get git info
os.chdir(repo_dir)

# set output directory
# output_base_dir = os.path.join(gdrive_mount, "MyDrive", "Dissertation")

In [ ]:

# ------------------------------------------------------------
# ------------------------------------------------------------
# 
#                   SCRIPT STARTS HERE
#
# ------------------------------------------------------------
# ------------------------------------------------------------


# calculate the hyper-parameters for GP on freeboard cover using a config
# - date(s)
# - window size
# - radius of inclusion
# - freeboard season

import json
import os
import numpy as np

from OptimalInterpolation import get_data_path, get_path
from OptimalInterpolation.sea_ice_freeboard import SeaIceFreeboard



In [ ]:



# #  change output_base_dir as needed
# output_base_dir = get_path("results")
output_base_dir = os.path.join(gdrive_mount, "MyDrive", "Dissertation")

# ---
# Parameters
# ---

grid_res = 50
coarse_grid_spacing = 1
incl_rad = 300
days_ahead = 4
days_behind = 4
min_inputs = 5
hold_out = None
bound_length_scales = True
prior_mean_method = "fyi_average"


# post process dict
# - let the std used in
post_process = {
    "clip_and_smooth": True,
    "smooth_method": "kernel",
    "std": coarse_grid_spacing,
    "vmax_map": {
        "ls_x": 2 * incl_rad * 1000,
        "ls_y": 2 * incl_rad * 1000,
        "ls_t": days_ahead + days_behind + 1,
        "kernel_variance": 0.1,
        "likelihood_variance": 0.05
    },
    "vmin_map": {
        "ls_x": 1,
        "ls_y": 1,
        "ls_t": 1e-6,
        "kernel_variance": 2e-6,
        "likelihood_variance": 2e-6
    }
}

# ---
# input config
# ---

config = {
    "dates": ["20181202", "20190102"], #["20190201", "20190301"]
    "output_dir": os.path.join(output_base_dir, "run_then_postprocess"),
    "incl_rad": incl_rad,
    "days_ahead": days_ahead,
    "days_behind": days_behind,
    "data_dir": "package",
    "season": "2018-2019",
    "grid_res": grid_res,
    "min_inputs": min_inputs,
    "verbose": 3,
    "engine": "GPflow",
    "kernel": "Matern32",
    "prior_mean_method": prior_mean_method,
    "hold_out": hold_out,
    "load_params": False,
    "pred_on_hold_out": False,
    "scale_inputs": True,
    "scale_outputs": False,
    "bound_length_scales": bound_length_scales,
    "append_to_file": True,
    "overwrite": False,
    "use_raw_data": False,
    "store_params": False,
    "predict_locations": ["center_only"],
    "skip_if_pred_exists": True,
    "store_loss": False
}

# ---
# parameters
# ---

print("using config:")
print(json.dumps(config, indent=4))

# extract parameters from config

season = config.get("season", "2018-2019")
assert season == "2018-2019", "only can handle data inputs from '2018-2019' season at the moment"

gres = config.get("grid_res", 50)

# pop some config parameters
data_dir = config.pop("data_dir")
verbose = config.pop("verbose", True)
dates = config.pop("dates")

# -----
# initialise SeaIceFreeboard object
# -----

sifb = SeaIceFreeboard(grid_res=f"{gres}km",
                       length_scale_name=["x", "y", "t"],
                       verbose=verbose,
                       rng_seed=None)

# ---
# read / load data
# ---

if data_dir == "package":
    data_dir = get_data_path()

assert os.path.exists(data_dir)

sifb.load_data(aux_data_dir=os.path.join(data_dir, "aux"),
               sat_data_dir=os.path.join(data_dir, "CS2S3_CPOM"),
               # raw_data_dir=os.path.join(data_dir, "RAW"),
               season=season)

# ---
# for each date increment over the model combinations
# ---

for date in dates:
    print("|" * 100)
    print(f"date: {date}")
    print("#" * 10)

    # ---
    # initial run
    # ---

    print("|*" * 50)
    print(f"initial run")

    # tmp_dir is where the results will be written to
    tmp_dir = sifb.make_temp_dir(incl_rad,
                                 days_ahead,
                                 days_behind,
                                 grid_res,
                                 season,
                                 coarse_grid_spacing,
                                 hold_out,
                                 bound_length_scales,
                                 prior_mean_method)

    # optimise - no post processing
    initial_file_suffix = ""
    sifb.run(date=date,
             optimise=True,
             coarse_grid_spacing=coarse_grid_spacing,
             file_suffix=initial_file_suffix,
             post_process=None,
             previous_results=None,
             tmp_dir=tmp_dir,
             **config)

    # ---
    # apply post processing
    # ---

    print("|*" * 50)
    print(f"post processing")
    print(f"date: {date}")
    print("#" * 10)

    # for post processing - need results from previous run
    previous_results = {
        'dir': os.path.join(config['output_dir'], tmp_dir),
        'suffix': initial_file_suffix
    }

    sifb.run(date=date,
             optimise=False,
             coarse_grid_spacing=1,
             file_suffix="_postproc",
             post_process=post_process,
             previous_results=previous_results,
             tmp_dir=tmp_dir,
             **config)
